<h1><center>Cassava Leaf Disease Classification</center></h1>
<center><img src="https://images.unsplash.com/photo-1536882240095-0379873feb4e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1051&q=80" width="60%"></center>

### In this notebook, I will use different keras available models to see how they affect the results.
### Here is the complete list of <a href='https://keras.io/api/applications/'>[keras available models] </a> . Please feel to fork this notebook and tweak it using different models/processings. I would be happy to discuss the results. <span style="color:red">Please do not forget to upvote</span>. 

### Version 6 updates: using Xception + adding dropout + lowering learning rate.
#### <span style="color:green">Please note that each Keras Application expects a specific kind of input preprocessing. For Xception, call tf.keras.applications.xception.preprocess_input on your inputs before passing them to the model.</span>.

# Part 1 - Training

In [ ]:
# Importing useful libraries

import pandas as pd 
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")

# ML tools 
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Conv2D, MaxPooling2D, Dropout, Conv2D,MaxPooling2D,GlobalAveragePooling2D
from keras.optimizers import Adam
from tensorflow.keras import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152V2, VGG16, InceptionResNetV2, EfficientNetB0,Xception, ResNet50, NASNetLarge
from keras import optimizers
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

# Visualization tools
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_target = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
display(df_target.head(3))
print(df_target.shape)
df_sample = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
display(df_target.head(3))
print(df_target.shape)

In [ ]:
df_target['label'] = df_target['label'].astype('str')
display(df_target.head())

In [ ]:
n_classes = 5
batch_size = 64
img_size = 224
n_epochs = 30

## ImageDataGenerator

The goal of applying data augmentation is to increase the generalizability of the model. Imagedatagenerator works with taking a batch of training examples and applying different random transformations to each image in the batch, for example: vertical flip, zoom, or rotation and pass the the batch with original or transformed images for the training. Therefore, we are not increasing the dataset size, but adding more variety to it. It should be noted that the training and validation folders should be seperate, otherwise, the augmentation will be implemented on the validation set too; which is not desirable. There is one way to perform the augmentation only on the training which is shown below by defining two different generators with passing validation_split in both. 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


image_generator = ImageDataGenerator(
        validation_split=0.15,
        horizontal_flip=True,
        vertical_flip = True,
        rotation_range=30,
        zoom_range = 0.25,
        shear_range = 0.15,
        fill_mode='nearest'
)

image_generator_valid = ImageDataGenerator(validation_split=0.15)
                                                   
train_generator = image_generator.flow_from_dataframe(
        dataframe = df_target,
        directory='../input/cassava-leaf-disease-classification/train_images',
        x_col = 'image_id',
        y_col = 'label',     
        target_size=(img_size, img_size),
        batch_size=batch_size,
        subset='training',
        class_mode='sparse') 

valid_generator=image_generator_valid.flow_from_dataframe(
    dataframe = df_target,
    directory='../input/cassava-leaf-disease-classification/train_images',
    x_col = 'image_id',
    y_col = 'label', 
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='sparse') 

Here are how images look like after passing through image generator. 

In [ ]:
for j in range(6):
    aug_images = [train_generator[0][0][j]/255 for i in range(6)]
    fig, axes = plt.subplots(1, 6, figsize=(24,24))
    axes = axes.flatten()
    for img, ax in zip(aug_images, axes):
        ax.imshow(img)
        ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
net = ResNet152V2(weights='imagenet', 
                  include_top = False, 
                  input_shape=(img_size, img_size, 3))
# net.trainable = False

In [ ]:
inp = keras.layers.Input([img_size, img_size, 3])
x = keras.applications.resnet_v2.preprocess_input(inp)
x = net(x)
x= GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
output = Dense(n_classes, activation='softmax')(x)
model = Model(inp, output)
model.summary()
model.compile(optimizers.Adam(lr=5e-4),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Defining callbacks to monitor the validation accuracy and save the model


rlr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.2, patience = 2, verbose = 0, 
                                min_delta = 1e-4, min_lr = 1e-6, mode = 'max')
        
ckp = ModelCheckpoint('model.h5',monitor = 'val_accuracy',
                      verbose = 0, save_best_only = True, mode = 'max')
        
es = EarlyStopping(monitor = 'val_accuracy', min_delta = 1e-4, patience = 5, mode = 'max', 
                    restore_best_weights = True, verbose = 0)


In [ ]:
history = model.fit_generator(generator=train_generator,                      
                    validation_data=valid_generator,                                       
                    epochs=n_epochs,
                    callbacks=[rlr,es,ckp],
                    verbose=2)
K.clear_session()

Here are the curves showing training and validation losses and accuracies over epochs. 

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
hist = pd.DataFrame(history.history)
fig, (ax1, ax2) = plt.subplots(figsize=(12,12),nrows=2, ncols=1)
hist['loss'].plot(ax=ax1,c='k',label='training loss')
hist['val_loss'].plot(ax=ax1,c='r',linestyle='--', label='validation loss')
ax1.legend()
hist['accuracy'].plot(ax=ax2,c='k',label='training accuracy')
hist['val_accuracy'].plot(ax=ax2,c='r',linestyle='--',label='validation accuracy')
ax2.legend()
plt.show()

### The model is saved using the checkpoint callback.
## For the results, please see the second notebook <a href='https://www.kaggle.com/sinamhd9/efficientnetb0-in-keras-part-2-inference/'> [Part 2]</a>

## Please upvote this notebook if you find it useful. Thanks!